In [1]:
!python traditional_features.py

Testing traditional feature extraction framework...
All required libraries available.
Created test audio: 32000 samples at 16000 Hz

Testing individual extractors...
MFCC features shape: (39, 63)
Expected dimension: 39
Mel spectrogram shape: (64, 63)
/usr/local/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=500
  warnings.warn(
Spectral features shape: (17, 63)

Testing complete pipeline...
Extracted features from 4 extractors:
  mfcc: (39,)
  mel_spectrogram: (64,)
  spectral_features: (17,)
  statistical_features: (14,)
Combined features shape: (134,)
Expected total dimension: 134

Testing post-processing...
Training array shape: (10, 134)
Fitting with features shape: (10, 134)
/usr/local/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=500
  warnings.warn(
Transforming features with shape: (1, 134)
Reshaped features for transformation: (

In [2]:
!python feature_adapter.py

Testing feature adapter system...
Traditional features shape: (13, 100)
Chaotic features shape: (8, 50)

Testing DimensionAdapter...
Dimension adapted: 100 -> 100

Testing TemporalAdapter...
Temporal adapted shape: (13, 100) -> (13,)

Testing NormalizationAdapter...
Normalized features - mean: 0.0000, std: 0.0000

Testing ModelSpecificAdapter...
Model adapted shape: (13,) -> (1, 13)

Testing complete pipeline...
Pipeline result shape: (100, 13) -> (100, 20)

Testing comparison adapters...
Created adapter: traditional_mlp
Created adapter: traditional_chaotic
Created adapter: chaotic_mlp
Created adapter: chaotic_chaotic

Pipeline statistics: {'pipeline_stats': {'total_adaptations': 1, 'adapter_stats': {'dimension_adapter': 1, 'normalization_adapter': 1, 'model_specific_adapter': 1}}, 'adapter_stats': {'dimension_adapter': {'total_adaptations': 2, 'input_shapes_seen': [(100, 13)], 'output_shapes_produced': [(100, 20)]}, 'normalization_adapter': {'total_adaptations': 2, 'input_shapes_seen'

In [8]:
!python chaotic_features.py

Testing Chaotic Feature Extractor...
Generated 5 Lorenz test signals
Created configuration successfully
/scratch/project_2003370/yueyao/Model/core/mlsa_extractor.py:39: UserWarning: Failed to import required modules: No module named 'chaos_utils'
  warnings.warn(f"Failed to import required modules: {e}")
Using real MLSAExtractor
/scratch/project_2003370/yueyao/Model/core/rqa_extractor.py:38: UserWarning: Failed to import required modules: No module named 'phase_space_reconstruction'
  warnings.warn(f"Failed to import required modules: {e}")
Using real RQAExtractor
✓ ChaoticFeatureExtractor created successfully!

Fitting chaotic feature extractor...
Fitting chaotic feature extractor on 3 signals...
/scratch/project_2003370/yueyao/Model/core/mlsa_extractor.py:715: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / np.sum(psd)
/scratch/project_2003370/yueyao/Model/core/mlsa_extractor.py:805: RuntimeWarning: invalid value encountered in divide
  signal_norm = (signal - 

In [13]:
!python test_traditional_features.py

Running comprehensive traditional features tests...
Available components:
  - Traditional features: True
  - Feature adapters: True
  - librosa: True
  - scikit-learn: True
  - PyTorch: True
  - TensorFlow: False

============================= test session starts ==============================
platform linux -- Python 3.12.10, pytest-8.4.1, pluggy-1.6.0 -- /usr/bin/python
cachedir: .pytest_cache
rootdir: /scratch/project_2003370/yueyao/Model/features
plugins: anyio-4.9.0, dash-3.1.0, jaxtyping-0.3.2
collected 60 items                                                             

test_traditional_features.py::TestTraditionalFeatureExtractors::test_mel_spectrogram_extractor_creation PASSED [  1%]
test_traditional_features.py::TestTraditionalFeatureExtractors::test_mel_spectrogram_extraction PASSED [  3%]
test_traditional_features.py::TestTraditionalFeatureExtractors::test_mel_spectrogram_different_signals PASSED [  5%]
test_traditional_features.py::TestTraditionalFeatureExtractors::test_

In [3]:
# debug_postprocessor.py
import numpy as np
import warnings
import importlib
import traditional_features as tf  # 或者 features.traditional_features，视你的包结构而定

# 1) 创建一个只含 MFCC 的 pipeline（与测试逻辑一致）
pipeline = tf.create_mfcc_pipeline(sample_rate=16000, n_mfcc=13, include_deltas=False, normalization='standard')
print("Pipeline created. Post-processor:", type(pipeline.post_processor))

# 2) 造三份训练音频（与测试类似）
sr = 16000
duration = 0.5
t = np.linspace(0, duration, int(sr * duration), endpoint=False)
base_signal = 0.5 * np.sin(2 * np.pi * 440 * t)
training_features = []
for i in range(3):
    noise = np.random.normal(0, 0.1, size=base_signal.shape)
    feat = pipeline.extract_and_combine(base_signal + noise)
    print(f"train sample {i} raw first 6 elems:", np.round(feat[:6], 4))
    training_features.append(feat)

training_array = np.array(training_features)
print("Training array shape:", training_array.shape)
print("Training array (first 3 rows):\n", np.round(training_array[:3, :], 6))

# 3) Fit post-processor (this triggers FeaturePostProcessor.fit)
pipeline.fit_post_processor(training_features)

pp = pipeline.post_processor
if pp is None:
    raise RuntimeError("No post_processor present!")

# 4) Inspect scaler internals (if present)
if pp.scaler is not None:
    print("Scaler present. n_features_in_ =", getattr(pp.scaler, "n_features_in_", None))
    print("Scaler mean_ (first 10):", np.round(pp.scaler.mean_[:10], 6))
    # scale_ may exist or scale_ = sqrt(var_)
    if hasattr(pp.scaler, "scale_"):
        print("Scaler scale_ (first 10):", np.round(pp.scaler.scale_[:10], 6))
    if hasattr(pp.scaler, "var_"):
        print("Scaler var_ (first 10):", np.round(pp.scaler.var_[:10], 6))
else:
    print("No scaler initialized in post-processor.")

# 5) Show what scaler.transform does to training samples
if pp.scaler is not None:
    transformed = pp.scaler.transform(training_array)
    print("Transformed training_array (first 3 rows):\n", np.round(transformed[:3, :], 6))
else:
    print("Skipping transform since no scaler.")

# 6) Now mimic pipeline.extract_and_combine + transform for a single sample
sample = pipeline.extract_and_combine(base_signal)  # a (13,) vector
print("Single sample (pre-transform) first 10:", np.round(sample[:10], 6))

transformed_single = pp.transform(sample.reshape(1, -1))
print("Post-processor.transform(single) returned shape:", transformed_single.shape)
print("Transformed single (flattened) first 10:", np.round(transformed_single.flatten()[:10], 6))
print("Mean of transformed single:", float(np.mean(transformed_single)))


Pipeline created. Post-processor: <class 'traditional_features.FeaturePostProcessor'>
train sample 0 raw first 6 elems: [  51.1043  176.5874  141.8762   65.5916   -8.3841 -114.3318]
train sample 1 raw first 6 elems: [  12.0782  169.7073  133.2236   45.917   -54.0375 -130.8698]
train sample 2 raw first 6 elems: [  -5.1652  154.2615  157.5664   78.288   -47.3611 -115.5242]
Training array shape: (3, 13)
Training array (first 3 rows):
 [[  51.104296  176.587383  141.876161   65.59156    -8.384064 -114.331846
  -207.06039  -190.713356 -162.048409 -102.638471  -49.202641    7.779992
   101.551441]
 [  12.078199  169.707321  133.223592   45.916977  -54.037536 -130.869794
  -175.275742 -188.133538 -158.589063  -85.908561  -23.607992   46.993122
   127.1893  ]
 [  -5.165164  154.261548  157.566361   78.287959  -47.361064 -115.524217
  -158.353059 -178.338542 -161.107044  -99.85623   -21.410925   26.680034
    95.91968 ]]
Fitting with features shape: (3, 13)
Scaler present. n_features_in_ = 13
S